# More contracts

In this tutorial, we will walk through several examples on how to use our BitML-to-ISPL compiler, and how to interpret the MCMAS output.

We put all required imports and constants in the following code block: 

In [1]:
from pathlib import Path
from bitml2mcmas.bitml.parser.parser import BitMLParser
from bitml2mcmas.compiler._private.terms import PrivateSecretValues, PublicSecretValues
from bitml2mcmas.compiler.core import Compiler
from bitml2mcmas.mcmas.ast import EvaluationRule
from bitml2mcmas.mcmas.boolcond import EqualTo, EnvironmentIdAtom, IntAtom, GreaterThanOrEqual, IdAtom
from bitml2mcmas.mcmas.formula import AtomicFormula, DiamondEventuallyFormula, AGFormula, ImpliesFormula, NotFormula, \
    AndFormula, OrFormula
from bitml2mcmas.mcmas.to_string import interpreted_system_to_string

## Propositions and Evaluation Rules

In this section, we define the propositions and their evaluation rules, which define the alphabet to express ATL specifications.

In [2]:
# Agent names
AGENT_A = "Agent_A"
AGENT_B = "Agent_B"
AGENT_M = "Agent_M"
AGENT_G = "Agent_G"

# State variable names
# A boolean keeping track whether participant A is ready to move to the next timestep
VAR_PART_A_IS_DONE = "part_A_is_done"
# A boolean keeping track whether participant B is ready to move to the next timestep
VAR_PART_B_IS_DONE = "part_B_is_done"
# An integer keeping track of the funds currently held by the contract
VAR_CONTRACT_FUNDS = "contract_funds"
# An integer keeping track of the total deposits held by participant A
VAR_PART_A_TOTAL_DEPOSITS = "part_A_total_deposits"
# An integer keeping track of the total deposits held by participant B
VAR_PART_B_TOTAL_DEPOSITS = "part_B_total_deposits"
# The private state of the secret 'a', an enumeration of {not_committed, valid, invalid}
VAR_PRIVATE_SECRET_A = "private_secret_a"
# The state of the public secret 'a', an enumeration of {not_committed, committed, valid}
VAR_PUBLIC_SECRET_A = "public_secret_a"
# The state of the private secret 'b', an enumeration of {not_committed, valid, invalid}
VAR_PRIVATE_SECRET_B = "private_secret_b"
# The state of the public secret 'b', an enumeration of {not_committed, committed, valid}
VAR_PUBLIC_SECRET_B = "public_secret_b"

# propositions and their evaluation rules (some of them are added by the compiler by default)
#####
# part_A_is_done: true iff the state variable 'part_A_is_done' is true
# Definition: part_A_is_done if (Environment.part_A_is_done = true);
PART_A_IS_DONE_PROP = "part_A_is_done"
PART_A_IS_DONE = AtomicFormula(PART_A_IS_DONE_PROP)
#####
# part_B_is_done: true iff the state variable 'part_B_is_done' is true
# Definition: part_B_is_done if (Environment.part_B_is_done = true);
PART_B_IS_DONE_PROP = "part_B_is_done"
PART_B_IS_DONE = AtomicFormula(PART_B_IS_DONE_PROP)
#####
# contract_is_initialized: true iff the contract is initialized or not in the current state
# Definition: contract_is_initialized if (Environment.contract_initialized = true);
CONTRACT_IS_INITIALIZED_PROP = "contract_is_initialized"
CONTRACT_IS_INITIALIZED = AtomicFormula(CONTRACT_IS_INITIALIZED_PROP)
#####
# contract_funds_are_zero: true iff there are no more funds deposited in the contracts
# Definition: contract_funds_are_zero if (Environment.contract_funds = 0);
CONTRACT_FUNDS_ARE_ZERO_PROP = "contract_funds_are_zero"
CONTRACT_FUNDS_ARE_ZERO = AtomicFormula(CONTRACT_FUNDS_ARE_ZERO_PROP)
CONTRACT_FUNDS_ARE_ZERO_ER = EvaluationRule(
    CONTRACT_FUNDS_ARE_ZERO_PROP, 
    EqualTo(EnvironmentIdAtom(VAR_CONTRACT_FUNDS), IntAtom(0))
)
#####
# timeout_1_has_expired: true iff the first timeout in the contract has expired
# Definition: timeout_1_has_expired if (Environment.time >= 1);
TIMEOUT_1_HAS_EXPIRED_PROP = "timeout_1_has_expired"
TIMEOUT_1_HAS_EXPIRED = AtomicFormula(TIMEOUT_1_HAS_EXPIRED_PROP)
#####
# part_A_total_deposits_is_1: the total of deposits held by participant A is equal to 1
# Definition: part_A_total_deposits_is_1 if (Environment.part_A_total_deposits = 1);
PART_A_TOTAL_DEPOSITS_IS_1_PROP = "part_A_total_deposits_is_1"
PART_A_TOTAL_DEPOSITS_IS_1 = AtomicFormula(PART_A_TOTAL_DEPOSITS_IS_1_PROP)
PART_A_TOTAL_DEPOSITS_IS_1_ER = EvaluationRule(
    PART_A_TOTAL_DEPOSITS_IS_1_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PART_A_TOTAL_DEPOSITS), IntAtom(1))
)
#####
# part_A_total_deposits_is_at_least_1: the total of deposits held by participant A is at least 1
# Definition: part_A_total_deposits_is_at_least_1 if (Environment.part_A_total_deposits >= 1);
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP = "part_A_total_deposits_is_at_least_1"
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1 = AtomicFormula(PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP)
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER = EvaluationRule(
    PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_A_TOTAL_DEPOSITS), IntAtom(1))
)
#####
# part_B_total_deposits_is_1: the total of deposits held by participant B is equal to 1
# Definition: part_B_total_deposits_is_1 if (Environment.part_B_total_deposits = 1);
PART_B_TOTAL_DEPOSITS_IS_1_PROP = "part_B_total_deposits_is_1"
PART_B_TOTAL_DEPOSITS_IS_1 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_1_PROP)
PART_B_TOTAL_DEPOSITS_IS_1_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_1_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(1))
)
#####
# part_B_total_deposits_is_at_least_1: the total of deposits held by participant B is at least 1
# Definition: part_B_total_deposits_is_at_least_1 if (Environment.part_B_total_deposits >= 1);
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP = "part_B_total_deposits_is_at_least_1"
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP)
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(1))
)
#####
# part_B_total_deposits_is_2: the total of deposits held by participant B is equal to 2
# Definition: part_B_total_deposits_is_2 if (Environment.part_B_total_deposits = 2);
PART_B_TOTAL_DEPOSITS_IS_2_PROP = "part_B_total_deposits_is_2"
PART_B_TOTAL_DEPOSITS_IS_2 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_2_PROP)
PART_B_TOTAL_DEPOSITS_IS_2_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_2_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(2))
)
#####
# part_A_total_deposits_is_at_least_9: the total of deposits held by participant A is at least 9
# Definition: part_A_total_deposits_is_at_least_9 if (Environment.part_A_total_deposits >= 9);
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP = "part_A_total_deposits_is_at_least_9"
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9 = AtomicFormula(PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP)
PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9_ER = EvaluationRule(
    PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_A_TOTAL_DEPOSITS), IntAtom(9))
)
#####
# part_B_total_deposits_is_at_least_9: the total of deposits held by participant B is at least 9
# Definition: part_B_total_deposits_is_at_least_9 if (Environment.part_B_total_deposits >= 9);
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP = "part_B_total_deposits_is_at_least_9"
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9 = AtomicFormula(PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP)
PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_ER = EvaluationRule(
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_B_TOTAL_DEPOSITS), IntAtom(9))
)
#####
# part_A_total_deposits_is_10: the total of deposits held by participant A is 10
# Definition: part_A_total_deposits_is_10 if (Environment.part_A_total_deposits = 10);
PART_A_TOTAL_DEPOSITS_IS_10_PROP = "part_A_total_deposits_is_10"
PART_A_TOTAL_DEPOSITS_IS_10 = AtomicFormula(PART_A_TOTAL_DEPOSITS_IS_10_PROP)
PART_A_TOTAL_DEPOSITS_IS_10_ER = EvaluationRule(
    PART_A_TOTAL_DEPOSITS_IS_10_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_A_TOTAL_DEPOSITS), IntAtom(10))
)
####
# private_secret_a_is_valid: the secret commitment for 'a' is valid
# Definition: private_secret_a_is_valid if (Agent_A.private_secret_a = valid);
PRIVATE_SECRET_A_IS_VALID_PROP = "private_secret_a_is_valid"
PRIVATE_SECRET_A_IS_VALID = AtomicFormula(PRIVATE_SECRET_A_IS_VALID_PROP)
PRIVATE_SECRET_A_IS_VALID_ER = EvaluationRule(
    PRIVATE_SECRET_A_IS_VALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PRIVATE_SECRET_A), IdAtom(PrivateSecretValues.VALID.value))
)
####
# private_secret_b_is_valid: the secret commitment for 'b' is valid
# Definition: private_secret_b_is_valid if (Agent_B.private_secret_b = valid);
PRIVATE_SECRET_B_IS_VALID_PROP = "private_secret_b_is_valid"
PRIVATE_SECRET_B_IS_VALID = AtomicFormula(PRIVATE_SECRET_B_IS_VALID_PROP)
PRIVATE_SECRET_B_IS_VALID_ER = EvaluationRule(
    PRIVATE_SECRET_B_IS_VALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PRIVATE_SECRET_B), IdAtom(PrivateSecretValues.VALID.value))
)
####
# private_secret_a_is_invalid: the secret commitment for 'a' is invalid
# Definition: private_secret_a_is_invalid if (Agent_A.private_secret_a = invalid);
PRIVATE_SECRET_A_IS_INVALID_PROP = "private_secret_a_is_invalid"
PRIVATE_SECRET_A_IS_INVALID = AtomicFormula(PRIVATE_SECRET_A_IS_INVALID_PROP)
PRIVATE_SECRET_A_IS_INVALID_ER = EvaluationRule(
    PRIVATE_SECRET_A_IS_INVALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PRIVATE_SECRET_A), IdAtom(PrivateSecretValues.INVALID.value))
)
####
# private_secret_b_is_invalid: the secret commitment for 'b' is invalid
# Definition: private_secret_b_is_invalid if (Agent_B.private_secret_b = invalid);
PRIVATE_SECRET_B_IS_INVALID_PROP = "private_secret_b_is_invalid"
PRIVATE_SECRET_B_IS_INVALID = AtomicFormula(PRIVATE_SECRET_B_IS_INVALID_PROP)
PRIVATE_SECRET_B_IS_INVALID_ER = EvaluationRule(
    PRIVATE_SECRET_B_IS_INVALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PRIVATE_SECRET_B), IdAtom(PrivateSecretValues.INVALID.value))
)
####
# public_secret_a_is_valid: the secret commitment for 'a' is valid
# Definition: public_secret_a_is_valid if (Environment.public_secret_a = valid);
PUBLIC_SECRET_A_IS_VALID_PROP = "public_secret_a_is_valid"
PUBLIC_SECRET_A_IS_VALID = AtomicFormula(PUBLIC_SECRET_A_IS_VALID_PROP)
PUBLIC_SECRET_A_IS_VALID_ER = EvaluationRule(
    PUBLIC_SECRET_A_IS_VALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PUBLIC_SECRET_A), IdAtom(PublicSecretValues.VALID.value))
)
####
# public_secret_b_is_valid: the secret commitment for 'b' is invalid
# Definition: public_secret_b_is_valid if (Environment.public_secret_b = invalid);
PUBLIC_SECRET_B_IS_VALID_PROP = "public_secret_b_is_valid"
PUBLIC_SECRET_B_IS_VALID = AtomicFormula(PUBLIC_SECRET_B_IS_VALID_PROP)
PUBLIC_SECRET_B_IS_VALID_ER = EvaluationRule(
    PUBLIC_SECRET_B_IS_VALID_PROP,
    EqualTo(EnvironmentIdAtom(VAR_PUBLIC_SECRET_B), IdAtom(PublicSecretValues.VALID.value))
)

## Timed Commitment

In this section, we will verify a BitML formalization of the _timed commitment_ protocol. 

First, we need to compile the BitML contract into an ISPL file. 

The BitML contract is the following:

In [3]:
bitml_timed_commitment = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")

(contract
  (pre
    (deposit "A" 1 "txA@0")
    (secret "A" a "0001a")
  )
  (choice
    (reveal (a) (withdraw "A"))
    (after 1 (withdraw "B"))
  )
)
"""



We will verify the following ${\rm ATL}$ specifications:

- Participant $A$ has a strategy to make sure the contract is liquidated: \\[ AG(\mathsf{contract\_is\_initialized} \to \langle\mathsf{A}\rangle(\mathsf{contract\_funds\_is\_zero})) \\]
- Participant $B$ has a strategy to make sure the contract is liquidated: \\[AG(\mathsf{contract\_is\_initialized} \to \langle\mathsf{B}\rangle(\mathsf{contract\_funds\_is\_zero}))\\]
- Participant $A$ has a strategy to recover his funds whenever he commits to a valid secret (wrong attempt): \\[ AG((A.\mathsf{private\_secret\_a} = \mathsf{valid}) \to \langle A \rangle F (\mathsf{part\_A\_total\_deposits} = 1)) \\]
- Participant $A$ has a strategy to recover his funds whenever he commits to a valid secret AND timeout has not expired: \\[AG((A.\mathsf{private\_secret\_a} = \mathsf{valid} \wedge \mathsf{time} < t \wedge \lnot\mathsf{A\_is\_done}) \to \langle A \rangle F (\mathsf{part\_A\_total\_deposits} = 1))\\]
- If $A$ is dishonest, then $B$ can always find a way to withdraw the penalty: \\[ AG(A.\mathsf{private\_secret\_a} = \mathsf{invalid}) \to \langle A\rangle F (\mathsf{part\_B\_total\_deposits}=1)) \\]

The Python code to produce the ISPL is:


In [4]:
# parse the contract
parser = BitMLParser()
contract = parser(bitml_timed_commitment)

# AG(contract_is_initialized A> <Agent_A>F(contract_funds_are_zero))
AGENT_A_CAN_ENFORCE_LIQUIDITY = AGFormula(
    ImpliesFormula(
        CONTRACT_IS_INITIALIZED,
        DiamondEventuallyFormula(AGENT_A, CONTRACT_FUNDS_ARE_ZERO),
    )
)
# AG(contract_is_initialized -> <Agent_B>F(contract_funds_are_zero))
AGENT_B_CAN_ENFORCE_LIQUIDITY = AGFormula(
    ImpliesFormula(
        CONTRACT_IS_INITIALIZED,
        DiamondEventuallyFormula(AGENT_B, CONTRACT_FUNDS_ARE_ZERO),
    )
)
# AG(private_secret_a_is_valid -> <Agent_B>F(contract_funds_are_zero))
AGENT_A_HONEST_CAN_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        PRIVATE_SECRET_A_IS_VALID,
        DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_1),
    )
)
# AG(
#   (private_secret_a_is_valid and !timeout_1_has_expired and !part_A_is_done) 
#   -> <Agent_B>F(contract_funds_are_zero))
AGENT_A_HONEST_CAN_RECOVER_FUNDS_IF_TIMEOUT_NOT_EXPIRED = AGFormula(
    ImpliesFormula(
        AndFormula(PRIVATE_SECRET_A_IS_VALID, 
                   AndFormula(
                       NotFormula(TIMEOUT_1_HAS_EXPIRED), 
                       NotFormula(PART_A_IS_DONE))),
        DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_1),
    )
)

# AG(private_secret_a_is_invalid -> <Agent_B>F(part_B_total_deposits_is_1))
IF_AGENT_A_IS_DISHONEST_AGENT_B_GETS_A_PENALTY = AGFormula(
    ImpliesFormula(
        PRIVATE_SECRET_A_IS_INVALID,
        DiamondEventuallyFormula(AGENT_B, PART_B_TOTAL_DEPOSITS_IS_1)
    )
)

formulae = [
    AGENT_A_CAN_ENFORCE_LIQUIDITY,
    AGENT_B_CAN_ENFORCE_LIQUIDITY,
    AGENT_A_HONEST_CAN_RECOVER_FUNDS,
    AGENT_A_HONEST_CAN_RECOVER_FUNDS_IF_TIMEOUT_NOT_EXPIRED,
    IF_AGENT_A_IS_DISHONEST_AGENT_B_GETS_A_PENALTY
]
evaluation_rules = [
    CONTRACT_FUNDS_ARE_ZERO_ER,
    PART_A_TOTAL_DEPOSITS_IS_1_ER,
    PART_B_TOTAL_DEPOSITS_IS_1_ER
]
compiler = Compiler(
    contract, formulae, evaluation_rules=evaluation_rules
)
interpreted_system = compiler.compile()
interpreted_system_str = interpreted_system_to_string(interpreted_system)
_ = Path("01-timed-commitment/contract.ispl").write_text(interpreted_system_str)

The output file `01-timed-commitment/contract.ispl` is pasted below:


```
Semantics=SingleAssignment;
Agent Environment
  Obsvars:
    time: 0..1;
    contract_funds: 0..1;
    part_A_total_deposits: 0..1;
    part_B_total_deposits: 0..1;
    public_secret_a: {committed, not_committed, valid};
    contract_initialized: boolean;
    status_node_0_withdraw: {disabled, enabled, executed};
    status_node_1_reveal: {disabled, enabled, executed};
    status_node_2_withdraw: {disabled, enabled, executed};
  end Obsvars
  Vars:
    part_A_is_done: boolean;
    part_B_is_done: boolean;
    last_action: {action_delay, action_schedule_part_A, action_schedule_part_B, unset};
  end Vars

  Actions = {delay, schedule_part_A, schedule_part_B};
  Protocol:
    (part_A_is_done = false): {schedule_part_A};
    (part_B_is_done = false): {schedule_part_B};
    Other: {delay};
  end Protocol
  Evolution:
    part_A_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_B_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_A_is_done = true if (((Action = schedule_part_A) and (Agent_A.Action = nop)) and (part_A_is_done = false));
    part_B_is_done = true if (((Action = schedule_part_B) and (Agent_B.Action = nop)) and (part_B_is_done = false));
    time = (time + 1) if (((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true))) and (time < 1));
    public_secret_a = committed if (((public_secret_a = not_committed) and ((Agent_A.Action = commit_valid_secret_a) or (Agent_A.Action = commit_invalid_secret_a))) and (Action = schedule_part_A));
    public_secret_a = valid if (((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)) and (public_secret_a = committed));
    contract_initialized = true if (((contract_initialized = false) and ((public_secret_a = committed) or (public_secret_a = valid))) and (((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    status_node_0_withdraw = executed if (((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)));
    status_node_0_withdraw = enabled if (((status_node_1_reveal = executed) or (((Agent_A.Action = exec_node_1_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_reveal) and (Action = schedule_part_B)))) and (status_node_0_withdraw = disabled));
    status_node_1_reveal = executed if (((Agent_A.Action = exec_node_1_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_reveal) and (Action = schedule_part_B)));
    status_node_1_reveal = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_1_reveal = disabled)) and ((public_secret_a = valid) or ((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)))) and (!((status_node_2_withdraw = executed) or (((Agent_A.Action = exec_node_2_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_withdraw) and (Action = schedule_part_B))))));
    status_node_1_reveal = disabled if ((status_node_2_withdraw = executed) or (((Agent_A.Action = exec_node_2_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    status_node_2_withdraw = executed if (((Agent_A.Action = exec_node_2_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_withdraw) and (Action = schedule_part_B)));
    status_node_2_withdraw = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_2_withdraw = disabled)) and ((time >= 1) or ((time = 0) and (Action = delay)))) and (!((status_node_1_reveal = executed) or (((Agent_A.Action = exec_node_1_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_reveal) and (Action = schedule_part_B))))));
    status_node_2_withdraw = disabled if ((status_node_1_reveal = executed) or (((Agent_A.Action = exec_node_1_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_reveal) and (Action = schedule_part_B))));
    last_action = action_schedule_part_B if (Action = schedule_part_B);
    last_action = action_schedule_part_A if (Action = schedule_part_A);
    last_action = action_delay if (Action = delay);
  end Evolution
end Agent

Agent Agent_A
  Lobsvars = {part_A_is_done};
  Vars:
    private_secret_a: {invalid, not_committed, valid};
  end Vars

  Actions = {commit_invalid_secret_a, commit_valid_secret_a, exec_node_0_withdraw, exec_node_1_reveal, exec_node_2_withdraw, initialize_contract, nop, reveal_secret_a};
  Protocol:
    (((Environment.contract_initialized = false) and (private_secret_a = not_committed)) and (Environment.part_A_is_done = false)): {commit_invalid_secret_a, commit_valid_secret_a, nop};
    (((private_secret_a = valid) and (Environment.public_secret_a = committed)) and (Environment.part_A_is_done = false)): {nop, reveal_secret_a};
    (((Environment.contract_initialized = false) and (Environment.part_A_is_done = false)) and ((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_reveal = enabled) and (Environment.part_A_is_done = false)): {exec_node_1_reveal, nop};
    ((Environment.status_node_2_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_2_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    private_secret_a = valid if ((((Action = commit_valid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
    private_secret_a = invalid if ((((Action = commit_invalid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
  end Evolution
end Agent
Agent Agent_B
  Lobsvars = {part_B_is_done};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {exec_node_0_withdraw, exec_node_1_reveal, exec_node_2_withdraw, initialize_contract, nop};
  Protocol:
    (((Environment.contract_initialized = false) and (Environment.part_B_is_done = false)) and ((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_reveal = enabled) and (Environment.part_B_is_done = false)): {exec_node_1_reveal, nop};
    ((Environment.status_node_2_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_2_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Evaluation
  part_A_is_scheduled if (Environment.last_action = action_schedule_part_A);
  part_B_is_scheduled if (Environment.last_action = action_schedule_part_B);
  time_progresses_forever if ((Environment.time = 1) or (Environment.last_action = action_delay));
  time_reaches_maximum if (Environment.time = 1);
  timeout_1_has_expired if (Environment.time >= 1);
  part_A_is_done if (Environment.part_A_is_done = true);
  part_B_is_done if (Environment.part_B_is_done = true);
  private_secret_a_is_valid if (Agent_A.private_secret_a = valid);
  private_secret_a_is_invalid if (Agent_A.private_secret_a = invalid);
  private_secret_a_is_not_committed if (Agent_A.private_secret_a = not_committed);
  public_secret_a_is_committed if (Environment.public_secret_a = committed);
  public_secret_a_is_not_committed if (Environment.public_secret_a = not_committed);
  public_secret_a_is_valid if (Environment.public_secret_a = valid);
  contract_is_initialized if (Environment.contract_initialized = true);
  node_0_withdraw_is_disabled if (Environment.status_node_0_withdraw = disabled);
  node_0_withdraw_is_enabled if (Environment.status_node_0_withdraw = enabled);
  node_0_withdraw_is_executed if (Environment.status_node_0_withdraw = executed);
  node_1_reveal_is_disabled if (Environment.status_node_1_reveal = disabled);
  node_1_reveal_is_enabled if (Environment.status_node_1_reveal = enabled);
  node_1_reveal_is_executed if (Environment.status_node_1_reveal = executed);
  node_2_withdraw_is_disabled if (Environment.status_node_2_withdraw = disabled);
  node_2_withdraw_is_enabled if (Environment.status_node_2_withdraw = enabled);
  node_2_withdraw_is_executed if (Environment.status_node_2_withdraw = executed);
  contract_funds_are_zero if (Environment.contract_funds = 0);
  part_A_total_deposits_is_1 if (Environment.part_A_total_deposits = 1);
  part_B_total_deposits_is_1 if (Environment.part_B_total_deposits = 1);
end Evaluation
InitStates
  ((((((((((((((Environment.time = 0) and (Environment.part_A_is_done = false)) and (Environment.part_B_is_done = false)) and (Environment.contract_funds = 1)) and (Environment.part_A_total_deposits = 0)) and (Environment.part_B_total_deposits = 0)) and (Environment.public_secret_a = not_committed)) and (Agent_A.private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.status_node_0_withdraw = disabled)) and (Environment.status_node_1_reveal = disabled)) and (Environment.status_node_2_withdraw = disabled)) and (Environment.last_action = unset)) and (Agent_B.dummy = false));
end InitStates
Groups
  Participants = {Agent_A, Agent_B};
  Env = {Environment};
  ParticipantsAndEnv = {Agent_A, Agent_B, Environment};
  Agent_A = {Agent_A};
  Agent_B = {Agent_B};
end Groups
Fairness
  part_A_is_scheduled;
  part_B_is_scheduled;
  time_progresses_forever;
end Fairness
Formulae
  (AG ((contract_is_initialized) -> ((<Agent_A>F(contract_funds_are_zero)))));
  (AG ((contract_is_initialized) -> ((<Agent_B>F(contract_funds_are_zero)))));
  (AG ((private_secret_a_is_valid) -> ((<Agent_A>F(part_A_total_deposits_is_1)))));
  (AG (((private_secret_a_is_valid and ((!timeout_1_has_expired) and (!part_A_is_done)))) -> ((<Agent_A>F(part_A_total_deposits_is_1)))));
  (AG ((private_secret_a_is_invalid) -> ((<Agent_B>F(part_B_total_deposits_is_1)))));
end Formulae
``` 

By calling MCMAS with the command:

```
./mcmas -atlk 1  docs/01-timed-commitment/contract.ispl
```

We get the following output:


```
************************************************************************
                       MCMAS v1.3.0 

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law. 

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./tests/bin/mcmas -atlk 1 docs/01-timed-commitment/contract.ispl

docs/01-timed-commitment/contract.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (AG (contract_is_initialized -> (<Agent_A>F contract_funds_are_zero))), is TRUE in the model
  Formula number 2: (AG (contract_is_initialized -> (<Agent_B>F contract_funds_are_zero))), is TRUE in the model
  Formula number 3: (AG (private_secret_a_is_valid -> (<Agent_A>F part_A_total_deposits_is_1))), is FALSE in the model
  Formula number 4: (AG ((private_secret_a_is_valid && ((! timeout_1_has_expired) && (! part_A_is_done))) -> (<Agent_A>F part_A_total_deposits_is_1))), is TRUE in the model
  Formula number 5: (AG (private_secret_a_is_invalid -> (<Agent_B>F part_B_total_deposits_is_1))), is TRUE in the model
done, 5 formulae successfully read and checked
execution time = 0.171
number of reachable states = 161
BDD memory in use = 10796592
```


## Mutual Timed Commitment

In this section, we will verify a BitML formalization of the _mutual timed commitment_ protocol. 

The BitML contract is the following:

In [5]:
bitml_mutual_timed_commitment = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")

(contract
  (pre
    (deposit "A" 1 "txA@0")
    (deposit "B" 1 "txB@0")
    (secret "A" a "0001a")
    (secret "B" b "0001b")
  )
  (choice
    (reveal (a) (choice
      (reveal (b) (split
        (1 -> (withdraw "A"))
        (1 -> (withdraw "B"))
      ))
      (after 2 (withdraw "A"))
    ))
    (after 1 (withdraw "B"))
  )
)
"""


We will verify the following ${\rm ATL}$ specifications:

- If participant $A$ publishes an invalid commit, then he cannot recover his funds: \\[AG((A.\mathsf{private\_secret\_a} = \mathsf{invalid}) \to \lnot \langle A \rangle F (\mathsf{part\_A\_total\_deposits} \ge 1))\\]
- If timeout $t$ has expired and commitment for secret $a$ is invalid, then $B$ can recover his funds plus $A$'s penalty without revealing its secret $b$:: \\[
    AG(
        (\mathsf{public\_secret\_b} \neq \mathsf{valid} \wedge A.\mathsf{private\_secret\_a}=\mathsf{invalid} \wedge \mathsf{time} \ge t) \to \langle B \rangle F (\mathsf{public\_secret\_b} \neq \mathsf{valid} \wedge \mathsf{part\_B\_total\_deposits} = 2))
\\]
- If both participants commit to their secrets, then it is guaranteed that $B$ has always a strategy to take at least $1$ bitcoin: \\[ AG((\mathsf{contract\_is\_initialized} \wedge \mathsf{private\_secret\_b} = \mathsf{valid} \wedge \mathsf{time} < t) \to \langle B\rangle F(\mathsf{part\_B\_total\_deposits}\ge 1) ) \\]

The Python code to produce the ISPL is:


In [6]:
# parse the contract
parser = BitMLParser()
contract = parser(bitml_mutual_timed_commitment)

# AG(contract_is_initialized -> <Agent_A>F(contract_funds_are_zero))
AGENT_A_CAN_ENFORCE_LIQUIDITY = AGFormula(
    ImpliesFormula(
        CONTRACT_IS_INITIALIZED,
        DiamondEventuallyFormula(AGENT_A, CONTRACT_FUNDS_ARE_ZERO),
    )
)
# AG(contract_is_initialized -> <Agent_B>F(contract_funds_are_zero))
AGENT_B_CAN_ENFORCE_LIQUIDITY = AGFormula(
    ImpliesFormula(
        CONTRACT_IS_INITIALIZED,
        DiamondEventuallyFormula(AGENT_B, CONTRACT_FUNDS_ARE_ZERO),
    )
)
# AG((private_secret_a_is_invalid) -> (!<Agent_A>F(part_A_total_deposits_is_at_least_1)))
IF_AGENT_A_IS_DISHONEST_HE_DOES_NOT_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        PRIVATE_SECRET_A_IS_INVALID,
        NotFormula(
            DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_1)
        )
    )
)

# AG((!public_secret_b_is_valid and private_secret_a_is_invalid and timeout_1_has_expired) 
#     -> <Agent_B>F(!public_secret_b_is_valid and part_B_total_deposits_is_2))
IF_AGENT_A_IS_DISHONEST_AND_TIMEOUT_1_EXPIRED_THEN_B_CAN_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        AndFormula(
            NotFormula(PUBLIC_SECRET_B_IS_VALID),
            AndFormula(
                PRIVATE_SECRET_A_IS_INVALID,
                TIMEOUT_1_HAS_EXPIRED
            )
        ),
        DiamondEventuallyFormula(AGENT_B, 
                                 AndFormula(
                                     NotFormula(PUBLIC_SECRET_B_IS_VALID),
                                     PART_B_TOTAL_DEPOSITS_IS_2
                                 )
        )
    )
)

# AG((contract_is_initialized and private_secret_b_is_valid and !timeout_1_has_expired) 
#    -> (<Agent_B>F(part_B_total_deposits_is_at_least_1)))
IF_BOTH_AGENT_A_AND_AGENT_B_HONEST_B_CAN_RECOVER_FUNDS = AGFormula(
    ImpliesFormula(
        AndFormula(
            CONTRACT_IS_INITIALIZED,
            AndFormula(
                PRIVATE_SECRET_B_IS_VALID,
                NotFormula(TIMEOUT_1_HAS_EXPIRED)
            )
        ),
        DiamondEventuallyFormula(AGENT_B, PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1),
    )
)

formulae = [
    AGENT_A_CAN_ENFORCE_LIQUIDITY,
    AGENT_B_CAN_ENFORCE_LIQUIDITY,
    IF_AGENT_A_IS_DISHONEST_AND_TIMEOUT_1_EXPIRED_THEN_B_CAN_RECOVER_FUNDS,
    IF_BOTH_AGENT_A_AND_AGENT_B_HONEST_B_CAN_RECOVER_FUNDS
]
evaluation_rules = [
    CONTRACT_FUNDS_ARE_ZERO_ER,
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER,
]
compiler = Compiler(
    contract, formulae, evaluation_rules=evaluation_rules
)
interpreted_system = compiler.compile()
interpreted_system_str = interpreted_system_to_string(interpreted_system)
_ = Path("02-mutual-timed-commitment/contract.ispl").write_text(interpreted_system_str)

The output file `02-mutual-timed-commitment/contract.ispl` is pasted below:

```
Semantics=SingleAssignment;
Agent Environment
  Obsvars:
    time: 0..2;
    contract_funds: 0..2;
    part_A_total_deposits: 0..2;
    part_B_total_deposits: 0..2;
    public_secret_a: {committed, not_committed, valid};
    public_secret_b: {committed, not_committed, valid};
    contract_initialized: boolean;
    status_node_0_withdraw: {disabled, enabled, executed};
    status_node_1_withdraw: {disabled, enabled, executed};
    status_node_2_split: {disabled, enabled, executed};
    status_node_3_reveal: {disabled, enabled, executed};
    status_node_4_withdraw: {disabled, enabled, executed};
    status_node_6_reveal: {disabled, enabled, executed};
    status_node_7_withdraw: {disabled, enabled, executed};
  end Obsvars
  Vars:
    part_A_is_done: boolean;
    part_B_is_done: boolean;
    last_action: {action_delay, action_schedule_part_A, action_schedule_part_B, unset};
  end Vars

  Actions = {delay, schedule_part_A, schedule_part_B};
  Protocol:
    (part_A_is_done = false): {schedule_part_A};
    (part_B_is_done = false): {schedule_part_B};
    Other: {delay};
  end Protocol
  Evolution:
    part_A_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_B_is_done = false if ((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true)));
    part_A_is_done = true if (((Action = schedule_part_A) and (Agent_A.Action = nop)) and (part_A_is_done = false));
    part_B_is_done = true if (((Action = schedule_part_B) and (Agent_B.Action = nop)) and (part_B_is_done = false));
    time = (time + 1) if (((Action = delay) and ((part_A_is_done = true) and (part_B_is_done = true))) and (time < 2));
    public_secret_a = committed if (((public_secret_a = not_committed) and ((Agent_A.Action = commit_valid_secret_a) or (Agent_A.Action = commit_invalid_secret_a))) and (Action = schedule_part_A));
    public_secret_a = valid if (((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)) and (public_secret_a = committed));
    public_secret_b = committed if (((public_secret_b = not_committed) and ((Agent_B.Action = commit_valid_secret_b) or (Agent_B.Action = commit_invalid_secret_b))) and (Action = schedule_part_B));
    public_secret_b = valid if (((Agent_B.Action = reveal_secret_b) and (Action = schedule_part_B)) and (public_secret_b = committed));
    contract_initialized = true if (((contract_initialized = false) and (((public_secret_a = committed) or (public_secret_a = valid)) and ((public_secret_b = committed) or (public_secret_b = valid)))) and (((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    status_node_0_withdraw = executed if (((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)));
    status_node_0_withdraw = enabled if (((status_node_2_split = executed) or (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)))) and (status_node_0_withdraw = disabled));
    contract_funds = (contract_funds - 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 1) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    status_node_1_withdraw = executed if (((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)));
    status_node_1_withdraw = enabled if (((status_node_2_split = executed) or (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)))) and (status_node_1_withdraw = disabled));
    status_node_2_split = executed if (((Agent_A.Action = exec_node_2_split) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_2_split) and (Action = schedule_part_B)));
    status_node_2_split = enabled if (((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B)))) and (status_node_2_split = disabled));
    status_node_3_reveal = executed if (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B)));
    status_node_3_reveal = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_3_reveal = disabled)) and ((public_secret_b = valid) or ((Agent_B.Action = reveal_secret_b) and (Action = schedule_part_B)))) and (!((status_node_4_withdraw = executed) or (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))))));
    status_node_3_reveal = disabled if ((status_node_4_withdraw = executed) or (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    status_node_4_withdraw = executed if (((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B)));
    status_node_4_withdraw = enabled if (((((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)))) and (status_node_4_withdraw = disabled)) and ((time >= 2) or ((time = 1) and (Action = delay)))) and (!((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))))));
    status_node_4_withdraw = disabled if ((status_node_3_reveal = executed) or (((Agent_A.Action = exec_node_3_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_3_reveal) and (Action = schedule_part_B))));
    status_node_6_reveal = executed if (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B)));
    status_node_6_reveal = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_6_reveal = disabled)) and ((public_secret_a = valid) or ((Agent_A.Action = reveal_secret_a) and (Action = schedule_part_A)))) and (!((status_node_7_withdraw = executed) or (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B))))));
    status_node_6_reveal = disabled if ((status_node_7_withdraw = executed) or (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 2) if (((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    status_node_7_withdraw = executed if (((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B)));
    status_node_7_withdraw = enabled if (((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_7_withdraw = disabled)) and ((time >= 1) or ((time = 0) and (Action = delay)))) and (!((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B))))));
    status_node_7_withdraw = disabled if ((status_node_6_reveal = executed) or (((Agent_A.Action = exec_node_6_reveal) and (Action = schedule_part_A)) or ((Agent_B.Action = exec_node_6_reveal) and (Action = schedule_part_B))));
    last_action = action_schedule_part_B if (Action = schedule_part_B);
    last_action = action_schedule_part_A if (Action = schedule_part_A);
    last_action = action_delay if (Action = delay);
  end Evolution
end Agent

Agent Agent_A
  Lobsvars = {part_A_is_done};
  Vars:
    private_secret_a: {invalid, not_committed, valid};
  end Vars

  Actions = {commit_invalid_secret_a, commit_valid_secret_a, exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_split, exec_node_3_reveal, exec_node_4_withdraw, exec_node_6_reveal, exec_node_7_withdraw, initialize_contract, nop, reveal_secret_a};
  Protocol:
    (((Environment.contract_initialized = false) and (private_secret_a = not_committed)) and (Environment.part_A_is_done = false)): {commit_invalid_secret_a, commit_valid_secret_a, nop};
    (((private_secret_a = valid) and (Environment.public_secret_a = committed)) and (Environment.part_A_is_done = false)): {nop, reveal_secret_a};
    (((Environment.contract_initialized = false) and (Environment.part_A_is_done = false)) and (((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid)) and ((Environment.public_secret_b = committed) or (Environment.public_secret_b = valid)))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_split = enabled) and (Environment.part_A_is_done = false)): {exec_node_2_split, nop};
    ((Environment.status_node_3_reveal = enabled) and (Environment.part_A_is_done = false)): {exec_node_3_reveal, nop};
    ((Environment.status_node_4_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_4_withdraw, nop};
    ((Environment.status_node_6_reveal = enabled) and (Environment.part_A_is_done = false)): {exec_node_6_reveal, nop};
    ((Environment.status_node_7_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_7_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    private_secret_a = valid if ((((Action = commit_valid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
    private_secret_a = invalid if ((((Action = commit_invalid_secret_a) and (private_secret_a = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_A));
  end Evolution
end Agent
Agent Agent_B
  Lobsvars = {part_B_is_done};
  Vars:
    private_secret_b: {invalid, not_committed, valid};
  end Vars

  Actions = {commit_invalid_secret_b, commit_valid_secret_b, exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_split, exec_node_3_reveal, exec_node_4_withdraw, exec_node_6_reveal, exec_node_7_withdraw, initialize_contract, nop, reveal_secret_b};
  Protocol:
    (((Environment.contract_initialized = false) and (private_secret_b = not_committed)) and (Environment.part_B_is_done = false)): {commit_invalid_secret_b, commit_valid_secret_b, nop};
    (((private_secret_b = valid) and (Environment.public_secret_b = committed)) and (Environment.part_B_is_done = false)): {nop, reveal_secret_b};
    (((Environment.contract_initialized = false) and (Environment.part_B_is_done = false)) and (((Environment.public_secret_a = committed) or (Environment.public_secret_a = valid)) and ((Environment.public_secret_b = committed) or (Environment.public_secret_b = valid)))): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_split = enabled) and (Environment.part_B_is_done = false)): {exec_node_2_split, nop};
    ((Environment.status_node_3_reveal = enabled) and (Environment.part_B_is_done = false)): {exec_node_3_reveal, nop};
    ((Environment.status_node_4_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_4_withdraw, nop};
    ((Environment.status_node_6_reveal = enabled) and (Environment.part_B_is_done = false)): {exec_node_6_reveal, nop};
    ((Environment.status_node_7_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_7_withdraw, nop};
    Other: {nop};
  end Protocol
  Evolution:
    private_secret_b = valid if ((((Action = commit_valid_secret_b) and (private_secret_b = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_B));
    private_secret_b = invalid if ((((Action = commit_invalid_secret_b) and (private_secret_b = not_committed)) and (Environment.contract_initialized = false)) and (Environment.Action = schedule_part_B));
  end Evolution
end Agent
Evaluation
  part_A_is_scheduled if (Environment.last_action = action_schedule_part_A);
  part_B_is_scheduled if (Environment.last_action = action_schedule_part_B);
  time_progresses_forever if ((Environment.time = 2) or (Environment.last_action = action_delay));
  time_reaches_maximum if (Environment.time = 2);
  timeout_1_has_expired if (Environment.time >= 1);
  timeout_2_has_expired if (Environment.time >= 2);
  part_A_is_done if (Environment.part_A_is_done = true);
  part_B_is_done if (Environment.part_B_is_done = true);
  private_secret_a_is_valid if (Agent_A.private_secret_a = valid);
  private_secret_a_is_invalid if (Agent_A.private_secret_a = invalid);
  private_secret_a_is_not_committed if (Agent_A.private_secret_a = not_committed);
  public_secret_a_is_committed if (Environment.public_secret_a = committed);
  public_secret_a_is_not_committed if (Environment.public_secret_a = not_committed);
  public_secret_a_is_valid if (Environment.public_secret_a = valid);
  private_secret_b_is_valid if (Agent_B.private_secret_b = valid);
  private_secret_b_is_invalid if (Agent_B.private_secret_b = invalid);
  private_secret_b_is_not_committed if (Agent_B.private_secret_b = not_committed);
  public_secret_b_is_committed if (Environment.public_secret_b = committed);
  public_secret_b_is_not_committed if (Environment.public_secret_b = not_committed);
  public_secret_b_is_valid if (Environment.public_secret_b = valid);
  contract_is_initialized if (Environment.contract_initialized = true);
  node_0_withdraw_is_disabled if (Environment.status_node_0_withdraw = disabled);
  node_0_withdraw_is_enabled if (Environment.status_node_0_withdraw = enabled);
  node_0_withdraw_is_executed if (Environment.status_node_0_withdraw = executed);
  node_1_withdraw_is_disabled if (Environment.status_node_1_withdraw = disabled);
  node_1_withdraw_is_enabled if (Environment.status_node_1_withdraw = enabled);
  node_1_withdraw_is_executed if (Environment.status_node_1_withdraw = executed);
  node_2_split_is_disabled if (Environment.status_node_2_split = disabled);
  node_2_split_is_enabled if (Environment.status_node_2_split = enabled);
  node_2_split_is_executed if (Environment.status_node_2_split = executed);
  node_3_reveal_is_disabled if (Environment.status_node_3_reveal = disabled);
  node_3_reveal_is_enabled if (Environment.status_node_3_reveal = enabled);
  node_3_reveal_is_executed if (Environment.status_node_3_reveal = executed);
  node_4_withdraw_is_disabled if (Environment.status_node_4_withdraw = disabled);
  node_4_withdraw_is_enabled if (Environment.status_node_4_withdraw = enabled);
  node_4_withdraw_is_executed if (Environment.status_node_4_withdraw = executed);
  node_6_reveal_is_disabled if (Environment.status_node_6_reveal = disabled);
  node_6_reveal_is_enabled if (Environment.status_node_6_reveal = enabled);
  node_6_reveal_is_executed if (Environment.status_node_6_reveal = executed);
  node_7_withdraw_is_disabled if (Environment.status_node_7_withdraw = disabled);
  node_7_withdraw_is_enabled if (Environment.status_node_7_withdraw = enabled);
  node_7_withdraw_is_executed if (Environment.status_node_7_withdraw = executed);
  contract_funds_are_zero if (Environment.contract_funds = 0);
  part_B_total_deposits_is_at_least_1 if (Environment.part_B_total_deposits >= 1);
end Evaluation
InitStates
  (((((((((((((((((((Environment.time = 0) and (Environment.part_A_is_done = false)) and (Environment.part_B_is_done = false)) and (Environment.contract_funds = 2)) and (Environment.part_A_total_deposits = 0)) and (Environment.part_B_total_deposits = 0)) and (Environment.public_secret_a = not_committed)) and (Environment.public_secret_b = not_committed)) and (Agent_A.private_secret_a = not_committed)) and (Agent_B.private_secret_b = not_committed)) and (Environment.contract_initialized = false)) and (Environment.status_node_0_withdraw = disabled)) and (Environment.status_node_1_withdraw = disabled)) and (Environment.status_node_2_split = disabled)) and (Environment.status_node_3_reveal = disabled)) and (Environment.status_node_4_withdraw = disabled)) and (Environment.status_node_6_reveal = disabled)) and (Environment.status_node_7_withdraw = disabled)) and (Environment.last_action = unset));
end InitStates
Groups
  Participants = {Agent_A, Agent_B};
  Env = {Environment};
  ParticipantsAndEnv = {Agent_A, Agent_B, Environment};
  Agent_A = {Agent_A};
  Agent_B = {Agent_B};
end Groups
Fairness
  part_A_is_scheduled;
  part_B_is_scheduled;
  time_progresses_forever;
end Fairness
Formulae
  (AG ((contract_is_initialized) -> ((<Agent_A>F(contract_funds_are_zero)))));
  (AG ((contract_is_initialized) -> ((<Agent_B>F(contract_funds_are_zero)))));
  (AG ((((!public_secret_b_is_valid) and (private_secret_a_is_invalid and timeout_1_has_expired))) -> ((<Agent_B>F(((!public_secret_b_is_valid) and part_B_total_deposits_is_at_least_1))))));
  (AG (((contract_is_initialized and (private_secret_b_is_valid and (!timeout_1_has_expired)))) -> ((<Agent_B>F(part_B_total_deposits_is_at_least_1)))));
end Formulae
```

By calling MCMAS with the command:

```
./mcmas -atlk 1  docs/02-mutual-timed-commitment/contract.ispl
```

We get the following output:


```
************************************************************************
                       MCMAS v1.3.0 

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law. 

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./tests/bin/mcmas -atlk 1 docs/02-mutual-timed-commitment/contract.ispl

docs/02-mutual-timed-commitment/contract.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (AG (contract_is_initialized -> (<Agent_A>F contract_funds_are_zero))), is TRUE in the model
  Formula number 2: (AG (contract_is_initialized -> (<Agent_B>F contract_funds_are_zero))), is TRUE in the model
  Formula number 3: (AG (((! public_secret_b_is_valid) && (private_secret_a_is_invalid && timeout_1_has_expired)) -> (<Agent_B>F ((! public_secret_b_is_valid) && part_B_total_deposits_is_at_least_1)))), is TRUE in the model
  Formula number 4: (AG ((contract_is_initialized && (private_secret_b_is_valid && (! timeout_1_has_expired))) -> (<Agent_B>F part_B_total_deposits_is_at_least_1))), is TRUE in the model
done, 4 formulae successfully read and checked
execution time = 1.537
number of reachable states = 1659
BDD memory in use = 14566192
```

By replacing `after 2` with `after 1` and repeating the compilation and the verification, you get that Formula 4 is FALSE.
 

## Escrow

In this section, we will verify a BitML formalization of an escrow contract.

The BitML contract is the following:


In [7]:
bitml_escrow = """
#lang bitml

(participant "A" "0a")
(participant "B" "0b")
(participant "M" "0e")

(contract
  (pre
    (deposit "A" 10 "txA@0")
  )
  (choice
    (auth "A" (withdraw "B"))
    (auth "B" (withdraw "A"))
    (auth "A" (split
      (1 -> (withdraw "M"))
      (9 -> (choice
        (auth "M" (withdraw "A"))
        (auth "M" (withdraw "B"))
      ))
    ))
    (auth "B" (split
      (1 -> (withdraw "M"))
      (9 -> (choice
        (auth "M" (withdraw "A"))
        (auth "M" (withdraw "B"))
      ))
    ))
  )
)
"""

We will verify the following ${\rm ATL}$ specifications:

- The contract is not liquid from $A$'s perspective, i.e. the following specification does not hold: \\[ \langle A \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

  Similarly for $B$ and $M$:

\\[ \langle B \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

\\[ \langle M \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

- However, if we take any subset of at least two players, the contract becomes liquid:

\\[ \langle A, B \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

\\[ \langle A, M \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

\\[ \langle B, M \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

- $A$ (resp. $B$) can guarantee that $B$ (resp. $A$) gets at least $9$ bitcoin (false):

\\[ \langle A \rangle F (\mathsf{part\_A\_total\_deposits\_is\_at\_least\_9}) \\]

\\[ \langle A \rangle F (\mathsf{part\_A\_total\_deposits\_is\_at\_least\_9}) \\]


- If the `Resolve` branch has been taken, then the mediator $M$ gets its commission (true)

\\[ 
AG((\mathsf{status\_node\_6\_split\_is\_executed} \vee \mathsf{status\_node\_11\_split\_is\_executed}) \to \langle M \rangle F (\mathsf{part\_M\_total\_deposits\_is\_at\_least\_1})) 
\\]


- If the `Resolve` branch has been taken, and no withdraw has been already authorized, then the mediator $M$ is able to both reward $A$ and $B$ (true):

\\[
\begin{aligned}
&AG((\\
&    \qquad (\mathsf{status\_node\_6\_split\_is\_executed} \vee \mathsf{status\_node\_11\_split\_is\_executed})\\
&    \qquad \wedge \lnot\mathsf{node\_3\_withdraw\_is\_authorized\_by\_M}\\
&    \qquad \wedge \lnot\mathsf{node\_4\_withdraw\_is\_authorized\_by\_M}\\  
&    \qquad \wedge \lnot\mathsf{node\_8\_withdraw\_is\_authorized\_by\_M}\\
&    \qquad \wedge \lnot\mathsf{node\_9\_withdraw\_is\_authorized\_by\_M})\\
&    \qquad \to (\\
&    \qquad\qquad\quad  \langle M\rangle F (\mathsf{part\_A\_total\_deposits\_is\_at\_least\_9}) \\
&    \qquad\qquad\wedge \langle M\rangle F (\mathsf{part\_B\_total\_deposits\_is\_at\_least\_9}) \\
))
\end{aligned}
\\]


The Python code to produce the ISPL is:

In [8]:
# parse the contract
parser = BitMLParser()
contract = parser(bitml_escrow)

AGENT_A_AND_AGENT_B = "Agent_A__Agent_B"
AGENT_A_AND_AGENT_M = "Agent_A__Agent_M"
AGENT_B_AND_AGENT_M = "Agent_B__Agent_M"

# <Agent_A>F(contract_funds_are_zero)
AGENT_A_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_A, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_B>F(contract_funds_are_zero)
AGENT_B_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_B, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_M>F(contract_funds_are_zero)
AGENT_M_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_M, CONTRACT_FUNDS_ARE_ZERO)

# <Agent_A,Agent_B>F(contract_funds_are_zero)
AGENT_A_AND_AGENT_B_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_A_AND_AGENT_B, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_A,Agent_M>F(contract_funds_are_zero)
AGENT_A_AND_AGENT_M_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_A_AND_AGENT_M, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_B,Agent_M>F(contract_funds_are_zero)
AGENT_B_AND_AGENT_M_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_B_AND_AGENT_M, CONTRACT_FUNDS_ARE_ZERO)

# <Agent_A>F(part_A_total_deposits_is_at_least_9)
AGENT_A_CAN_GET_AT_LEAST_9 = DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9)
# <Agent_A>F(part_A_total_deposits_is_at_least_9)
AGENT_B_CAN_GET_AT_LEAST_9 = DiamondEventuallyFormula(AGENT_B, PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9)

# AG((status_node_6_split_is_executed | status_node_11_split_is_executed) 
#     -> <M>F(part_M_total_deposits_is_at_least_1))
VAR_PART_M_TOTAL_DEPOSITS = "part_M_total_deposits"
PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP = "part_M_total_deposits_is_at_least_1"
PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1 = AtomicFormula(PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP)
PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER = EvaluationRule(
    PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1_PROP,
    GreaterThanOrEqual(EnvironmentIdAtom(VAR_PART_M_TOTAL_DEPOSITS), IntAtom(1))
)
IF_RESOLVE_BRANCH_TAKEN_THEN_MEDIATOR_CAN_GET_COMMISSION = AGFormula(
    ImpliesFormula(
        AtomicFormula("node_6_split_is_executed") | AtomicFormula("node_11_split_is_executed"),
        DiamondEventuallyFormula(AGENT_M, PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1),
    )
)

# AG( ( (status_node_6_split_is_executed | status_node_11_split_is_executed)
#       & !node_3_withdraw_is_authorized_by_M 
#       & !node_4_withdraw_is_authorized_by_M 
#       & !node_8_withdraw_is_authorized_by_M 
#       & !node_9_withdraw_is_authorized_by_M)
#       -> (<M>F(part_A_total_deposits_is_at_least_9)
#           <M>F(part_B_total_deposits_is_at_least_9))
# )
IF_RESOLVE_BRANCH_TAKEN_THEN_MEDIATOR_CAN_GIVE_FUNDS_TO_A_OR_B = AGFormula(
    ImpliesFormula(
        (AtomicFormula("node_6_split_is_executed") | AtomicFormula("node_11_split_is_executed"))
        & ~AtomicFormula("node_3_withdraw_is_authorized_by_M")
        & ~AtomicFormula("node_4_withdraw_is_authorized_by_M")
        & ~AtomicFormula("node_8_withdraw_is_authorized_by_M")
        & ~AtomicFormula("node_9_withdraw_is_authorized_by_M"),
        AndFormula(
            DiamondEventuallyFormula(AGENT_M, PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9),
            DiamondEventuallyFormula(AGENT_M, PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9),
        )
    )
)

formulae = [
    AGENT_A_CAN_ENFORCE_LIQUIDITY,
    AGENT_B_CAN_ENFORCE_LIQUIDITY,
    AGENT_M_CAN_ENFORCE_LIQUIDITY,
    AGENT_A_AND_AGENT_B_CAN_ENFORCE_LIQUIDITY,
    AGENT_A_AND_AGENT_M_CAN_ENFORCE_LIQUIDITY,
    AGENT_B_AND_AGENT_M_CAN_ENFORCE_LIQUIDITY,
    AGENT_A_CAN_GET_AT_LEAST_9,
    AGENT_B_CAN_GET_AT_LEAST_9,
    IF_RESOLVE_BRANCH_TAKEN_THEN_MEDIATOR_CAN_GET_COMMISSION,
    IF_RESOLVE_BRANCH_TAKEN_THEN_MEDIATOR_CAN_GIVE_FUNDS_TO_A_OR_B
]
evaluation_rules = [
    CONTRACT_FUNDS_ARE_ZERO_ER,
    PART_A_TOTAL_DEPOSITS_IS_AT_LEAST_9_ER,
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_ER,
    PART_M_TOTAL_DEPOSITS_IS_AT_LEAST_1_ER
]
compiler = Compiler(
    contract, formulae, evaluation_rules=evaluation_rules
)
interpreted_system = compiler.compile()
interpreted_system_str = interpreted_system_to_string(interpreted_system)
_ = Path("03-escrow/contract.ispl").write_text(interpreted_system_str)

The output file `03-escrow/contract.ispl` is pasted below:

```
Semantics=SingleAssignment;
Agent Environment
  Obsvars:
    contract_funds: 0..10;
    part_A_total_deposits: 0..10;
    part_M_total_deposits: 0..10;
    part_B_total_deposits: 0..10;
    contract_initialized: boolean;
    node_0_withdraw_authorized_by_part_A: boolean;
    status_node_0_withdraw: {disabled, enabled, executed};
    node_1_withdraw_authorized_by_part_B: boolean;
    status_node_1_withdraw: {disabled, enabled, executed};
    status_node_2_withdraw: {disabled, enabled, executed};
    node_3_withdraw_authorized_by_part_M: boolean;
    status_node_3_withdraw: {disabled, enabled, executed};
    node_4_withdraw_authorized_by_part_M: boolean;
    status_node_4_withdraw: {disabled, enabled, executed};
    node_6_split_authorized_by_part_A: boolean;
    status_node_6_split: {disabled, enabled, executed};
    status_node_7_withdraw: {disabled, enabled, executed};
    node_8_withdraw_authorized_by_part_M: boolean;
    status_node_8_withdraw: {disabled, enabled, executed};
    node_9_withdraw_authorized_by_part_M: boolean;
    status_node_9_withdraw: {disabled, enabled, executed};
    node_11_split_authorized_by_part_B: boolean;
    status_node_11_split: {disabled, enabled, executed};
  end Obsvars
  Vars:
    last_action: {action_schedule_part_A, action_schedule_part_B, action_schedule_part_M, unset};
  end Vars

  Actions = {schedule_part_A, schedule_part_B, schedule_part_M};
  Protocol:
    Other: {schedule_part_A, schedule_part_B, schedule_part_M};
  end Protocol
  Evolution:
    contract_initialized = true if ((contract_initialized = false) and ((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_M) and (Agent_M.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))));
    contract_funds = (contract_funds - 10) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_0_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 10) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_0_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    status_node_0_withdraw = executed if ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)));
    node_0_withdraw_authorized_by_part_A = true if ((Agent_A.Action = authorize_node_0_withdraw) and (Action = schedule_part_A));
    status_node_0_withdraw = enabled if ((((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_M) and (Agent_M.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_0_withdraw = disabled)) and ((node_0_withdraw_authorized_by_part_A = true) or ((Agent_A.Action = authorize_node_0_withdraw) and (Action = schedule_part_A)))) and (((!((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B))))) and (!((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))))) and (!((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))))));
    status_node_0_withdraw = disabled if ((((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)))) or ((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B))))) or ((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))));
    contract_funds = (contract_funds - 10) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_1_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 10) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_1_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    status_node_1_withdraw = executed if ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)));
    node_1_withdraw_authorized_by_part_B = true if ((Agent_B.Action = authorize_node_1_withdraw) and (Action = schedule_part_B));
    status_node_1_withdraw = enabled if ((((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_M) and (Agent_M.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_1_withdraw = disabled)) and ((node_1_withdraw_authorized_by_part_B = true) or ((Agent_B.Action = authorize_node_1_withdraw) and (Action = schedule_part_B)))) and (((!((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B))))) and (!((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))))) and (!((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))))));
    status_node_1_withdraw = disabled if ((((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)))) or ((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B))))) or ((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))));
    contract_funds = (contract_funds - 1) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_2_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    part_M_total_deposits = (part_M_total_deposits + 1) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_2_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    status_node_2_withdraw = executed if ((((Agent_A.Action = exec_node_2_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_2_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_2_withdraw) and (Action = schedule_part_B)));
    status_node_2_withdraw = enabled if (((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))) and (status_node_2_withdraw = disabled));
    contract_funds = (contract_funds - 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_3_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_3_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_3_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_3_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_3_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_3_withdraw)));
    status_node_3_withdraw = executed if ((((Agent_A.Action = exec_node_3_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_3_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_3_withdraw) and (Action = schedule_part_B)));
    node_3_withdraw_authorized_by_part_M = true if ((Agent_M.Action = authorize_node_3_withdraw) and (Action = schedule_part_M));
    status_node_3_withdraw = enabled if (((((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))) and (status_node_3_withdraw = disabled)) and ((node_3_withdraw_authorized_by_part_M = true) or ((Agent_M.Action = authorize_node_3_withdraw) and (Action = schedule_part_M)))) and (!((status_node_4_withdraw = executed) or ((((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_4_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))))));
    status_node_3_withdraw = disabled if ((status_node_4_withdraw = executed) or ((((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_4_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_4_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_4_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_withdraw)));
    status_node_4_withdraw = executed if ((((Agent_A.Action = exec_node_4_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_4_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_4_withdraw) and (Action = schedule_part_B)));
    node_4_withdraw_authorized_by_part_M = true if ((Agent_M.Action = authorize_node_4_withdraw) and (Action = schedule_part_M));
    status_node_4_withdraw = enabled if (((((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))) and (status_node_4_withdraw = disabled)) and ((node_4_withdraw_authorized_by_part_M = true) or ((Agent_M.Action = authorize_node_4_withdraw) and (Action = schedule_part_M)))) and (!((status_node_3_withdraw = executed) or ((((Agent_A.Action = exec_node_3_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_3_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_3_withdraw) and (Action = schedule_part_B))))));
    status_node_4_withdraw = disabled if ((status_node_3_withdraw = executed) or ((((Agent_A.Action = exec_node_3_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_3_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_3_withdraw) and (Action = schedule_part_B))));
    status_node_6_split = executed if ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)));
    node_6_split_authorized_by_part_A = true if ((Agent_A.Action = authorize_node_6_split) and (Action = schedule_part_A));
    status_node_6_split = enabled if ((((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_M) and (Agent_M.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_6_split = disabled)) and ((node_6_split_authorized_by_part_A = true) or ((Agent_A.Action = authorize_node_6_split) and (Action = schedule_part_A)))) and (((!((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B))))) and (!((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)))))) and (!((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))))));
    status_node_6_split = disabled if ((((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)))) or ((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B))))) or ((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))));
    contract_funds = (contract_funds - 1) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_7_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    part_M_total_deposits = (part_M_total_deposits + 1) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_7_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_7_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_7_withdraw)));
    status_node_7_withdraw = executed if ((((Agent_A.Action = exec_node_7_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_7_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_7_withdraw) and (Action = schedule_part_B)));
    status_node_7_withdraw = enabled if (((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))) and (status_node_7_withdraw = disabled));
    contract_funds = (contract_funds - 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_8_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_8_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_8_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_8_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_8_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_8_withdraw)));
    status_node_8_withdraw = executed if ((((Agent_A.Action = exec_node_8_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_8_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_8_withdraw) and (Action = schedule_part_B)));
    node_8_withdraw_authorized_by_part_M = true if ((Agent_M.Action = authorize_node_8_withdraw) and (Action = schedule_part_M));
    status_node_8_withdraw = enabled if (((((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))) and (status_node_8_withdraw = disabled)) and ((node_8_withdraw_authorized_by_part_M = true) or ((Agent_M.Action = authorize_node_8_withdraw) and (Action = schedule_part_M)))) and (!((status_node_9_withdraw = executed) or ((((Agent_A.Action = exec_node_9_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_9_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_9_withdraw) and (Action = schedule_part_B))))));
    status_node_8_withdraw = disabled if ((status_node_9_withdraw = executed) or ((((Agent_A.Action = exec_node_9_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_9_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_9_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_9_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_9_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_9_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 9) if ((((Action = schedule_part_A) and (Agent_A.Action = exec_node_9_withdraw)) or ((Action = schedule_part_M) and (Agent_M.Action = exec_node_9_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_9_withdraw)));
    status_node_9_withdraw = executed if ((((Agent_A.Action = exec_node_9_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_9_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_9_withdraw) and (Action = schedule_part_B)));
    node_9_withdraw_authorized_by_part_M = true if ((Agent_M.Action = authorize_node_9_withdraw) and (Action = schedule_part_M));
    status_node_9_withdraw = enabled if (((((status_node_11_split = executed) or ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)))) and (status_node_9_withdraw = disabled)) and ((node_9_withdraw_authorized_by_part_M = true) or ((Agent_M.Action = authorize_node_9_withdraw) and (Action = schedule_part_M)))) and (!((status_node_8_withdraw = executed) or ((((Agent_A.Action = exec_node_8_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_8_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_8_withdraw) and (Action = schedule_part_B))))));
    status_node_9_withdraw = disabled if ((status_node_8_withdraw = executed) or ((((Agent_A.Action = exec_node_8_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_8_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_8_withdraw) and (Action = schedule_part_B))));
    status_node_11_split = executed if ((((Agent_A.Action = exec_node_11_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_11_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_11_split) and (Action = schedule_part_B)));
    node_11_split_authorized_by_part_B = true if ((Agent_B.Action = authorize_node_11_split) and (Action = schedule_part_B));
    status_node_11_split = enabled if ((((((((Action = schedule_part_A) and (Agent_A.Action = initialize_contract)) or ((Action = schedule_part_M) and (Agent_M.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_11_split = disabled)) and ((node_11_split_authorized_by_part_B = true) or ((Agent_B.Action = authorize_node_11_split) and (Action = schedule_part_B)))) and (((!((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B))))) and (!((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)))))) and (!((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))))));
    status_node_11_split = disabled if ((((status_node_0_withdraw = executed) or ((((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_0_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)))) or ((status_node_1_withdraw = executed) or ((((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_1_withdraw) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B))))) or ((status_node_6_split = executed) or ((((Agent_A.Action = exec_node_6_split) and (Action = schedule_part_A)) or ((Agent_M.Action = exec_node_6_split) and (Action = schedule_part_M))) or ((Agent_B.Action = exec_node_6_split) and (Action = schedule_part_B)))));
    last_action = action_schedule_part_B if (Action = schedule_part_B);
    last_action = action_schedule_part_A if (Action = schedule_part_A);
    last_action = action_schedule_part_M if (Action = schedule_part_M);
  end Evolution
end Agent

Agent Agent_A
  Lobsvars = {};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {authorize_node_0_withdraw, authorize_node_6_split, exec_node_0_withdraw, exec_node_11_split, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_withdraw, exec_node_4_withdraw, exec_node_6_split, exec_node_7_withdraw, exec_node_8_withdraw, exec_node_9_withdraw, initialize_contract, nop};
  Protocol:
    (Environment.contract_initialized = false): {initialize_contract, nop};
    (Environment.status_node_0_withdraw = enabled): {exec_node_0_withdraw, nop};
    ((Environment.node_0_withdraw_authorized_by_part_A = false) and (Environment.contract_initialized = true)): {authorize_node_0_withdraw, nop};
    (Environment.status_node_1_withdraw = enabled): {exec_node_1_withdraw, nop};
    (Environment.status_node_2_withdraw = enabled): {exec_node_2_withdraw, nop};
    (Environment.status_node_3_withdraw = enabled): {exec_node_3_withdraw, nop};
    (Environment.status_node_4_withdraw = enabled): {exec_node_4_withdraw, nop};
    (Environment.status_node_6_split = enabled): {exec_node_6_split, nop};
    ((Environment.node_6_split_authorized_by_part_A = false) and (Environment.contract_initialized = true)): {authorize_node_6_split, nop};
    (Environment.status_node_7_withdraw = enabled): {exec_node_7_withdraw, nop};
    (Environment.status_node_8_withdraw = enabled): {exec_node_8_withdraw, nop};
    (Environment.status_node_9_withdraw = enabled): {exec_node_9_withdraw, nop};
    (Environment.status_node_11_split = enabled): {exec_node_11_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Agent Agent_M
  Lobsvars = {};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {authorize_node_3_withdraw, authorize_node_4_withdraw, authorize_node_8_withdraw, authorize_node_9_withdraw, exec_node_0_withdraw, exec_node_11_split, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_withdraw, exec_node_4_withdraw, exec_node_6_split, exec_node_7_withdraw, exec_node_8_withdraw, exec_node_9_withdraw, initialize_contract, nop};
  Protocol:
    (Environment.contract_initialized = false): {initialize_contract, nop};
    (Environment.status_node_0_withdraw = enabled): {exec_node_0_withdraw, nop};
    (Environment.status_node_1_withdraw = enabled): {exec_node_1_withdraw, nop};
    (Environment.status_node_2_withdraw = enabled): {exec_node_2_withdraw, nop};
    (Environment.status_node_3_withdraw = enabled): {exec_node_3_withdraw, nop};
    ((Environment.node_3_withdraw_authorized_by_part_M = false) and (Environment.contract_initialized = true)): {authorize_node_3_withdraw, nop};
    (Environment.status_node_4_withdraw = enabled): {exec_node_4_withdraw, nop};
    ((Environment.node_4_withdraw_authorized_by_part_M = false) and (Environment.contract_initialized = true)): {authorize_node_4_withdraw, nop};
    (Environment.status_node_6_split = enabled): {exec_node_6_split, nop};
    (Environment.status_node_7_withdraw = enabled): {exec_node_7_withdraw, nop};
    (Environment.status_node_8_withdraw = enabled): {exec_node_8_withdraw, nop};
    ((Environment.node_8_withdraw_authorized_by_part_M = false) and (Environment.contract_initialized = true)): {authorize_node_8_withdraw, nop};
    (Environment.status_node_9_withdraw = enabled): {exec_node_9_withdraw, nop};
    ((Environment.node_9_withdraw_authorized_by_part_M = false) and (Environment.contract_initialized = true)): {authorize_node_9_withdraw, nop};
    (Environment.status_node_11_split = enabled): {exec_node_11_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Agent Agent_B
  Lobsvars = {};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {authorize_node_11_split, authorize_node_1_withdraw, exec_node_0_withdraw, exec_node_11_split, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_withdraw, exec_node_4_withdraw, exec_node_6_split, exec_node_7_withdraw, exec_node_8_withdraw, exec_node_9_withdraw, initialize_contract, nop};
  Protocol:
    (Environment.contract_initialized = false): {initialize_contract, nop};
    (Environment.status_node_0_withdraw = enabled): {exec_node_0_withdraw, nop};
    (Environment.status_node_1_withdraw = enabled): {exec_node_1_withdraw, nop};
    ((Environment.node_1_withdraw_authorized_by_part_B = false) and (Environment.contract_initialized = true)): {authorize_node_1_withdraw, nop};
    (Environment.status_node_2_withdraw = enabled): {exec_node_2_withdraw, nop};
    (Environment.status_node_3_withdraw = enabled): {exec_node_3_withdraw, nop};
    (Environment.status_node_4_withdraw = enabled): {exec_node_4_withdraw, nop};
    (Environment.status_node_6_split = enabled): {exec_node_6_split, nop};
    (Environment.status_node_7_withdraw = enabled): {exec_node_7_withdraw, nop};
    (Environment.status_node_8_withdraw = enabled): {exec_node_8_withdraw, nop};
    (Environment.status_node_9_withdraw = enabled): {exec_node_9_withdraw, nop};
    (Environment.status_node_11_split = enabled): {exec_node_11_split, nop};
    ((Environment.node_11_split_authorized_by_part_B = false) and (Environment.contract_initialized = true)): {authorize_node_11_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Evaluation
  part_A_is_scheduled if (Environment.last_action = action_schedule_part_A);
  part_M_is_scheduled if (Environment.last_action = action_schedule_part_M);
  part_B_is_scheduled if (Environment.last_action = action_schedule_part_B);
  contract_is_initialized if (Environment.contract_initialized = true);
  node_0_withdraw_is_disabled if (Environment.status_node_0_withdraw = disabled);
  node_0_withdraw_is_enabled if (Environment.status_node_0_withdraw = enabled);
  node_0_withdraw_is_executed if (Environment.status_node_0_withdraw = executed);
  node_0_withdraw_is_authorized_by_A if (Environment.node_0_withdraw_authorized_by_part_A = true);
  node_1_withdraw_is_disabled if (Environment.status_node_1_withdraw = disabled);
  node_1_withdraw_is_enabled if (Environment.status_node_1_withdraw = enabled);
  node_1_withdraw_is_executed if (Environment.status_node_1_withdraw = executed);
  node_1_withdraw_is_authorized_by_B if (Environment.node_1_withdraw_authorized_by_part_B = true);
  node_2_withdraw_is_disabled if (Environment.status_node_2_withdraw = disabled);
  node_2_withdraw_is_enabled if (Environment.status_node_2_withdraw = enabled);
  node_2_withdraw_is_executed if (Environment.status_node_2_withdraw = executed);
  node_3_withdraw_is_disabled if (Environment.status_node_3_withdraw = disabled);
  node_3_withdraw_is_enabled if (Environment.status_node_3_withdraw = enabled);
  node_3_withdraw_is_executed if (Environment.status_node_3_withdraw = executed);
  node_3_withdraw_is_authorized_by_M if (Environment.node_3_withdraw_authorized_by_part_M = true);
  node_4_withdraw_is_disabled if (Environment.status_node_4_withdraw = disabled);
  node_4_withdraw_is_enabled if (Environment.status_node_4_withdraw = enabled);
  node_4_withdraw_is_executed if (Environment.status_node_4_withdraw = executed);
  node_4_withdraw_is_authorized_by_M if (Environment.node_4_withdraw_authorized_by_part_M = true);
  node_6_split_is_disabled if (Environment.status_node_6_split = disabled);
  node_6_split_is_enabled if (Environment.status_node_6_split = enabled);
  node_6_split_is_executed if (Environment.status_node_6_split = executed);
  node_6_split_is_authorized_by_A if (Environment.node_6_split_authorized_by_part_A = true);
  node_7_withdraw_is_disabled if (Environment.status_node_7_withdraw = disabled);
  node_7_withdraw_is_enabled if (Environment.status_node_7_withdraw = enabled);
  node_7_withdraw_is_executed if (Environment.status_node_7_withdraw = executed);
  node_8_withdraw_is_disabled if (Environment.status_node_8_withdraw = disabled);
  node_8_withdraw_is_enabled if (Environment.status_node_8_withdraw = enabled);
  node_8_withdraw_is_executed if (Environment.status_node_8_withdraw = executed);
  node_8_withdraw_is_authorized_by_M if (Environment.node_8_withdraw_authorized_by_part_M = true);
  node_9_withdraw_is_disabled if (Environment.status_node_9_withdraw = disabled);
  node_9_withdraw_is_enabled if (Environment.status_node_9_withdraw = enabled);
  node_9_withdraw_is_executed if (Environment.status_node_9_withdraw = executed);
  node_9_withdraw_is_authorized_by_M if (Environment.node_9_withdraw_authorized_by_part_M = true);
  node_11_split_is_disabled if (Environment.status_node_11_split = disabled);
  node_11_split_is_enabled if (Environment.status_node_11_split = enabled);
  node_11_split_is_executed if (Environment.status_node_11_split = executed);
  node_11_split_is_authorized_by_B if (Environment.node_11_split_authorized_by_part_B = true);
  contract_funds_are_zero if (Environment.contract_funds = 0);
  part_A_total_deposits_is_at_least_9 if (Environment.part_A_total_deposits >= 9);
  part_B_total_deposits_is_at_least_9 if (Environment.part_B_total_deposits >= 9);
  part_M_total_deposits_is_at_least_1 if (Environment.part_M_total_deposits >= 1);
end Evaluation
InitStates
  (((((((((((((((((((((((((((Environment.contract_funds = 10) and (Environment.part_A_total_deposits = 0)) and (Environment.part_M_total_deposits = 0)) and (Environment.part_B_total_deposits = 0)) and (Environment.contract_initialized = false)) and (Environment.node_0_withdraw_authorized_by_part_A = false)) and (Environment.status_node_0_withdraw = disabled)) and (Environment.node_1_withdraw_authorized_by_part_B = false)) and (Environment.status_node_1_withdraw = disabled)) and (Environment.status_node_2_withdraw = disabled)) and (Environment.node_3_withdraw_authorized_by_part_M = false)) and (Environment.status_node_3_withdraw = disabled)) and (Environment.node_4_withdraw_authorized_by_part_M = false)) and (Environment.status_node_4_withdraw = disabled)) and (Environment.node_6_split_authorized_by_part_A = false)) and (Environment.status_node_6_split = disabled)) and (Environment.status_node_7_withdraw = disabled)) and (Environment.node_8_withdraw_authorized_by_part_M = false)) and (Environment.status_node_8_withdraw = disabled)) and (Environment.node_9_withdraw_authorized_by_part_M = false)) and (Environment.status_node_9_withdraw = disabled)) and (Environment.node_11_split_authorized_by_part_B = false)) and (Environment.status_node_11_split = disabled)) and (Environment.last_action = unset)) and (Agent_M.dummy = false)) and (Agent_A.dummy = false)) and (Agent_B.dummy = false));
end InitStates
Groups
  Participants = {Agent_A, Agent_B, Agent_M};
  Env = {Environment};
  ParticipantsAndEnv = {Agent_A, Agent_B, Agent_M, Environment};
  Agent_M = {Agent_M};
  Agent_A = {Agent_A};
  Agent_B = {Agent_B};
  Agent_A__Agent_M = {Agent_A, Agent_M};
  Agent_B__Agent_M = {Agent_B, Agent_M};
  Agent_A__Agent_B = {Agent_A, Agent_B};
end Groups
Fairness
  part_A_is_scheduled;
  part_M_is_scheduled;
  part_B_is_scheduled;
end Fairness
Formulae
  (<Agent_A>F(contract_funds_are_zero));
  (<Agent_B>F(contract_funds_are_zero));
  (<Agent_M>F(contract_funds_are_zero));
  (<Agent_A__Agent_B>F(contract_funds_are_zero));
  (<Agent_A__Agent_M>F(contract_funds_are_zero));
  (<Agent_B__Agent_M>F(contract_funds_are_zero));
  (<Agent_A>F(part_A_total_deposits_is_at_least_9));
  (<Agent_B>F(part_B_total_deposits_is_at_least_9));
  (AG (((node_6_split_is_executed or node_11_split_is_executed)) -> ((<Agent_M>F(part_M_total_deposits_is_at_least_1)))));
  (AG (((((((node_6_split_is_executed or node_11_split_is_executed) and (!node_3_withdraw_is_authorized_by_M)) and (!node_4_withdraw_is_authorized_by_M)) and (!node_8_withdraw_is_authorized_by_M)) and (!node_9_withdraw_is_authorized_by_M))) -> (((<Agent_M>F(part_A_total_deposits_is_at_least_9)) and (<Agent_M>F(part_B_total_deposits_is_at_least_9))))));
end Formulae
```

By calling MCMAS with the command:

```
./mcmas -atlk 1  docs/03-escrow/contract.ispl
```

We get the following output:


```
************************************************************************
                       MCMAS v1.3.0 

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law. 

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./tests/bin/mcmas -atlk 1 docs/03-escrow/contract.ispl

docs/03-escrow/contract.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (<Agent_A>F contract_funds_are_zero), is FALSE in the model
  Formula number 2: (<Agent_B>F contract_funds_are_zero), is FALSE in the model
  Formula number 3: (<Agent_M>F contract_funds_are_zero), is FALSE in the model
  Formula number 4: (<Agent_A__Agent_B>F contract_funds_are_zero), is TRUE in the model
  Formula number 5: (<Agent_A__Agent_M>F contract_funds_are_zero), is TRUE in the model
  Formula number 6: (<Agent_B__Agent_M>F contract_funds_are_zero), is TRUE in the model
  Formula number 7: (<Agent_A>F part_A_total_deposits_is_at_least_9), is FALSE in the model
  Formula number 8: (<Agent_B>F part_B_total_deposits_is_at_least_9), is FALSE in the model
  Formula number 9: (AG ((node_6_split_is_executed || node_11_split_is_executed) -> (<Agent_M>F part_M_total_deposits_is_at_least_1))), is TRUE in the model
  Formula number 10: (AG ((((((node_6_split_is_executed || node_11_split_is_executed) && (! node_3_withdraw_is_authorized_by_M)) && (! node_4_withdraw_is_authorized_by_M)) && (! node_8_withdraw_is_authorized_by_M)) && (! node_9_withdraw_is_authorized_by_M)) -> ((<Agent_M>F part_A_total_deposits_is_at_least_9) && (<Agent_M>F part_B_total_deposits_is_at_least_9)))), is TRUE in the model
done, 10 formulae successfully read and checked
execution time = 4.731
number of reachable states = 4612
BDD memory in use = 29765232
```

## Zero-Coupon Bond

In this section, we will verify a BitML formalization of a zero-coupon bond contract.

The BitML contract is the following:

In [9]:
bitml_zcb = """
#lang bitml

(participant "A" "00a")
(participant "B" "00b")
(participant "G" "00c")


(contract
  (pre
    (deposit "A" 9 "txA@0")
    (vol-deposit "B" txb1 10 "txB@0")
    (deposit "G" 10 "txG@0"))

  (split
    (9  -> (withdraw "B"))
    (10 -> (choice
      ; B pays the bond and G recovers his security deposit
      (after 1 (put (txb1) (split (10 -> (withdraw "A")) (10 -> (withdraw "G")))))
      ; guarantors pays the bond to A
      (after 2 (withdraw "A"))
    )))
)
"""

We will verify the following ${\rm ATL}$ specifications:

- The contract is liquid from a single agent's perspective: 

\\[ \langle A \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]  

\\[ \langle B \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

\\[ \langle M \rangle F (\mathsf{contract\_funds\_are\_zero}) \\]

- we can check that $A$ and $G$ can cooperatively make sure $A$ gets its investment back, plus interests:

\\[ 
\langle A, G\rangle F (\mathsf{part\_A\_total\_deposits\_is\_10})
\\]

- The bank can take at least $9$ bitcoin: 
 
\\[
\langle B\rangle F (\mathsf{part\_B\_total\_deposits\_is\_at\_least\_9})
\\]

- Neither $B$ nor $G$ can prevent $A$ from withdrawing funds:

\\[
\quad \lnot  \langle B\rangle G (\lnot\langle A \rangle F \mathsf{part\_A\_total\_deposits\_is\_10})\\
\wedge \lnot \langle G\rangle G (\lnot \langle A\rangle F \mathsf{part\_A\_total\_deposits\_is\_10})
\\]

The Python code to produce the ISPL is:

In [10]:
# parse the contract
parser = BitMLParser()
contract = parser(bitml_zcb)

AGENT_A_AND_AGENT_G = "Agent_A__Agent_G"

# <Agent_A>F(contract_funds_are_zero)
AGENT_A_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_A, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_B>F(contract_funds_are_zero)
AGENT_B_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_B, CONTRACT_FUNDS_ARE_ZERO)
# <Agent_G>F(contract_funds_are_zero)
AGENT_G_CAN_ENFORCE_LIQUIDITY = DiamondEventuallyFormula(AGENT_G, CONTRACT_FUNDS_ARE_ZERO)

# <Agent_A, Agent_G>F(part_A_total_deposits_is_10)
AGENT_A_AND_AGENT_G_CAN_MAKE_A_GET_10 = DiamondEventuallyFormula(AGENT_A_AND_AGENT_G, PART_A_TOTAL_DEPOSITS_IS_10)
# <Agent_B>F(part_B_total_deposits_is_at_least_9)
AGENT_B_CAN_GET_AT_LEAST_9 = DiamondEventuallyFormula(AGENT_B, PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9)

CANNOT_PREVENT_A_FROM_WITHDRAWING_FUNDS = AndFormula(
    ~DiamondEventuallyFormula(AGENT_B, ~DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_10)),
    ~DiamondEventuallyFormula(AGENT_B, ~DiamondEventuallyFormula(AGENT_A, PART_A_TOTAL_DEPOSITS_IS_10)),
)

formulae = [
    AGENT_A_CAN_ENFORCE_LIQUIDITY,
    AGENT_B_CAN_ENFORCE_LIQUIDITY,
    AGENT_G_CAN_ENFORCE_LIQUIDITY,
    AGENT_A_AND_AGENT_G_CAN_MAKE_A_GET_10,
    AGENT_B_CAN_GET_AT_LEAST_9
    
]
evaluation_rules = [
    CONTRACT_FUNDS_ARE_ZERO_ER,
    PART_A_TOTAL_DEPOSITS_IS_10_ER,
    PART_B_TOTAL_DEPOSITS_IS_AT_LEAST_9_ER,
]
compiler = Compiler(
    contract, formulae, evaluation_rules=evaluation_rules
)
interpreted_system = compiler.compile()
interpreted_system_str = interpreted_system_to_string(interpreted_system)
_ = Path("04-zero-coupon-bond/contract.ispl").write_text(interpreted_system_str)

The output file `04-zero-coupon-bond/contract.ispl` is pasted below:

```
Semantics=SingleAssignment;
Agent Environment
  Obsvars:
    time: 0..2;
    contract_funds: 0..29;
    part_G_total_deposits: 0..29;
    part_A_total_deposits: 0..29;
    part_B_total_deposits: 0..29;
    spent_deposit_txb1: boolean;
    contract_initialized: boolean;
    status_node_0_withdraw: {disabled, enabled, executed};
    status_node_1_withdraw: {disabled, enabled, executed};
    status_node_2_withdraw: {disabled, enabled, executed};
    status_node_3_split: {disabled, enabled, executed};
    status_node_4_put: {disabled, enabled, executed};
    status_node_5_withdraw: {disabled, enabled, executed};
    status_node_7_split: {disabled, enabled, executed};
  end Obsvars
  Vars:
    part_G_is_done: boolean;
    part_A_is_done: boolean;
    part_B_is_done: boolean;
    last_action: {action_delay, action_schedule_part_A, action_schedule_part_B, action_schedule_part_G, unset};
  end Vars

  Actions = {delay, schedule_part_A, schedule_part_B, schedule_part_G};
  Protocol:
    (part_G_is_done = false): {schedule_part_G};
    (part_A_is_done = false): {schedule_part_A};
    (part_B_is_done = false): {schedule_part_B};
    Other: {delay};
  end Protocol
  Evolution:
    part_G_is_done = false if ((Action = delay) and (((part_G_is_done = true) and (part_A_is_done = true)) and (part_B_is_done = true)));
    part_A_is_done = false if ((Action = delay) and (((part_G_is_done = true) and (part_A_is_done = true)) and (part_B_is_done = true)));
    part_B_is_done = false if ((Action = delay) and (((part_G_is_done = true) and (part_A_is_done = true)) and (part_B_is_done = true)));
    part_G_is_done = true if (((Action = schedule_part_G) and (Agent_G.Action = nop)) and (part_G_is_done = false));
    part_A_is_done = true if (((Action = schedule_part_A) and (Agent_A.Action = nop)) and (part_A_is_done = false));
    part_B_is_done = true if (((Action = schedule_part_B) and (Agent_B.Action = nop)) and (part_B_is_done = false));
    time = (time + 1) if (((Action = delay) and (((part_G_is_done = true) and (part_A_is_done = true)) and (part_B_is_done = true))) and (time < 2));
    contract_initialized = true if ((contract_initialized = false) and ((((Action = schedule_part_G) and (Agent_G.Action = initialize_contract)) or ((Action = schedule_part_A) and (Agent_A.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))));
    contract_funds = (contract_funds - 9) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_0_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    part_B_total_deposits = (part_B_total_deposits + 9) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_0_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_0_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_0_withdraw)));
    status_node_0_withdraw = executed if ((((Agent_G.Action = exec_node_0_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_0_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_0_withdraw) and (Action = schedule_part_B)));
    status_node_0_withdraw = enabled if (((status_node_7_split = executed) or ((((Agent_G.Action = exec_node_7_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_7_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_7_split) and (Action = schedule_part_B)))) and (status_node_0_withdraw = disabled));
    contract_funds = (contract_funds - 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_1_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_1_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_1_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_1_withdraw)));
    status_node_1_withdraw = executed if ((((Agent_G.Action = exec_node_1_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_1_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_1_withdraw) and (Action = schedule_part_B)));
    status_node_1_withdraw = enabled if (((status_node_3_split = executed) or ((((Agent_G.Action = exec_node_3_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_3_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_3_split) and (Action = schedule_part_B)))) and (status_node_1_withdraw = disabled));
    contract_funds = (contract_funds - 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_2_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    part_G_total_deposits = (part_G_total_deposits + 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_2_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_2_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_2_withdraw)));
    status_node_2_withdraw = executed if ((((Agent_G.Action = exec_node_2_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_2_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_2_withdraw) and (Action = schedule_part_B)));
    status_node_2_withdraw = enabled if (((status_node_3_split = executed) or ((((Agent_G.Action = exec_node_3_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_3_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_3_split) and (Action = schedule_part_B)))) and (status_node_2_withdraw = disabled));
    status_node_3_split = executed if ((((Agent_G.Action = exec_node_3_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_3_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_3_split) and (Action = schedule_part_B)));
    status_node_3_split = enabled if (((status_node_4_put = executed) or ((((Agent_G.Action = exec_node_4_put) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_4_put) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_4_put) and (Action = schedule_part_B)))) and (status_node_3_split = disabled));
    status_node_4_put = executed if ((((Agent_G.Action = exec_node_4_put) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_4_put) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_4_put) and (Action = schedule_part_B)));
    contract_funds = (contract_funds + 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_4_put)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_put))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_put)));
    spent_deposit_txb1 = true if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_4_put)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_put))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_put)));
    part_B_total_deposits = (part_B_total_deposits - 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_4_put)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_4_put))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_4_put)));
    status_node_4_put = enabled if ((((((status_node_7_split = executed) or ((((Agent_G.Action = exec_node_7_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_7_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_7_split) and (Action = schedule_part_B)))) and (status_node_4_put = disabled)) and (spent_deposit_txb1 = false)) and ((time >= 1) or ((time = 0) and (Action = delay)))) and (!((status_node_5_withdraw = executed) or ((((Agent_G.Action = exec_node_5_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_5_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_5_withdraw) and (Action = schedule_part_B))))));
    status_node_4_put = disabled if ((status_node_5_withdraw = executed) or ((((Agent_G.Action = exec_node_5_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_5_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_5_withdraw) and (Action = schedule_part_B))));
    contract_funds = (contract_funds - 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_5_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_5_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_5_withdraw)));
    part_A_total_deposits = (part_A_total_deposits + 10) if ((((Action = schedule_part_G) and (Agent_G.Action = exec_node_5_withdraw)) or ((Action = schedule_part_A) and (Agent_A.Action = exec_node_5_withdraw))) or ((Action = schedule_part_B) and (Agent_B.Action = exec_node_5_withdraw)));
    status_node_5_withdraw = executed if ((((Agent_G.Action = exec_node_5_withdraw) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_5_withdraw) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_5_withdraw) and (Action = schedule_part_B)));
    status_node_5_withdraw = enabled if (((((status_node_7_split = executed) or ((((Agent_G.Action = exec_node_7_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_7_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_7_split) and (Action = schedule_part_B)))) and (status_node_5_withdraw = disabled)) and ((time >= 2) or ((time = 1) and (Action = delay)))) and (!((status_node_4_put = executed) or ((((Agent_G.Action = exec_node_4_put) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_4_put) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_4_put) and (Action = schedule_part_B))))));
    status_node_5_withdraw = disabled if ((status_node_4_put = executed) or ((((Agent_G.Action = exec_node_4_put) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_4_put) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_4_put) and (Action = schedule_part_B))));
    status_node_7_split = executed if ((((Agent_G.Action = exec_node_7_split) and (Action = schedule_part_G)) or ((Agent_A.Action = exec_node_7_split) and (Action = schedule_part_A))) or ((Agent_B.Action = exec_node_7_split) and (Action = schedule_part_B)));
    status_node_7_split = enabled if ((((((Action = schedule_part_G) and (Agent_G.Action = initialize_contract)) or ((Action = schedule_part_A) and (Agent_A.Action = initialize_contract))) or ((Action = schedule_part_B) and (Agent_B.Action = initialize_contract))) or (contract_initialized = true)) and (status_node_7_split = disabled));
    last_action = action_schedule_part_B if (Action = schedule_part_B);
    last_action = action_schedule_part_A if (Action = schedule_part_A);
    last_action = action_schedule_part_G if (Action = schedule_part_G);
    last_action = action_delay if (Action = delay);
  end Evolution
end Agent

Agent Agent_G
  Lobsvars = {part_G_is_done};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_split, exec_node_4_put, exec_node_5_withdraw, exec_node_7_split, initialize_contract, nop};
  Protocol:
    ((Environment.contract_initialized = false) and (Environment.part_G_is_done = false)): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_G_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_G_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_withdraw = enabled) and (Environment.part_G_is_done = false)): {exec_node_2_withdraw, nop};
    ((Environment.status_node_3_split = enabled) and (Environment.part_G_is_done = false)): {exec_node_3_split, nop};
    ((Environment.status_node_4_put = enabled) and (Environment.part_G_is_done = false)): {exec_node_4_put, nop};
    ((Environment.status_node_5_withdraw = enabled) and (Environment.part_G_is_done = false)): {exec_node_5_withdraw, nop};
    ((Environment.status_node_7_split = enabled) and (Environment.part_G_is_done = false)): {exec_node_7_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Agent Agent_A
  Lobsvars = {part_A_is_done};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_split, exec_node_4_put, exec_node_5_withdraw, exec_node_7_split, initialize_contract, nop};
  Protocol:
    ((Environment.contract_initialized = false) and (Environment.part_A_is_done = false)): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_2_withdraw, nop};
    ((Environment.status_node_3_split = enabled) and (Environment.part_A_is_done = false)): {exec_node_3_split, nop};
    ((Environment.status_node_4_put = enabled) and (Environment.part_A_is_done = false)): {exec_node_4_put, nop};
    ((Environment.status_node_5_withdraw = enabled) and (Environment.part_A_is_done = false)): {exec_node_5_withdraw, nop};
    ((Environment.status_node_7_split = enabled) and (Environment.part_A_is_done = false)): {exec_node_7_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Agent Agent_B
  Lobsvars = {part_B_is_done};
  Vars:
    dummy: boolean;
  end Vars

  Actions = {exec_node_0_withdraw, exec_node_1_withdraw, exec_node_2_withdraw, exec_node_3_split, exec_node_4_put, exec_node_5_withdraw, exec_node_7_split, initialize_contract, nop};
  Protocol:
    ((Environment.contract_initialized = false) and (Environment.part_B_is_done = false)): {initialize_contract, nop};
    ((Environment.status_node_0_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_0_withdraw, nop};
    ((Environment.status_node_1_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_1_withdraw, nop};
    ((Environment.status_node_2_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_2_withdraw, nop};
    ((Environment.status_node_3_split = enabled) and (Environment.part_B_is_done = false)): {exec_node_3_split, nop};
    ((Environment.status_node_4_put = enabled) and (Environment.part_B_is_done = false)): {exec_node_4_put, nop};
    ((Environment.status_node_5_withdraw = enabled) and (Environment.part_B_is_done = false)): {exec_node_5_withdraw, nop};
    ((Environment.status_node_7_split = enabled) and (Environment.part_B_is_done = false)): {exec_node_7_split, nop};
    Other: {nop};
  end Protocol
  Evolution:
    dummy = false if (dummy = false);
  end Evolution
end Agent
Evaluation
  part_G_is_scheduled if (Environment.last_action = action_schedule_part_G);
  part_A_is_scheduled if (Environment.last_action = action_schedule_part_A);
  part_B_is_scheduled if (Environment.last_action = action_schedule_part_B);
  time_progresses_forever if ((Environment.time = 2) or (Environment.last_action = action_delay));
  time_reaches_maximum if (Environment.time = 2);
  timeout_1_has_expired if (Environment.time >= 1);
  timeout_2_has_expired if (Environment.time >= 2);
  part_G_is_done if (Environment.part_G_is_done = true);
  part_A_is_done if (Environment.part_A_is_done = true);
  part_B_is_done if (Environment.part_B_is_done = true);
  contract_is_initialized if (Environment.contract_initialized = true);
  node_0_withdraw_is_disabled if (Environment.status_node_0_withdraw = disabled);
  node_0_withdraw_is_enabled if (Environment.status_node_0_withdraw = enabled);
  node_0_withdraw_is_executed if (Environment.status_node_0_withdraw = executed);
  node_1_withdraw_is_disabled if (Environment.status_node_1_withdraw = disabled);
  node_1_withdraw_is_enabled if (Environment.status_node_1_withdraw = enabled);
  node_1_withdraw_is_executed if (Environment.status_node_1_withdraw = executed);
  node_2_withdraw_is_disabled if (Environment.status_node_2_withdraw = disabled);
  node_2_withdraw_is_enabled if (Environment.status_node_2_withdraw = enabled);
  node_2_withdraw_is_executed if (Environment.status_node_2_withdraw = executed);
  node_3_split_is_disabled if (Environment.status_node_3_split = disabled);
  node_3_split_is_enabled if (Environment.status_node_3_split = enabled);
  node_3_split_is_executed if (Environment.status_node_3_split = executed);
  node_4_put_is_disabled if (Environment.status_node_4_put = disabled);
  node_4_put_is_enabled if (Environment.status_node_4_put = enabled);
  node_4_put_is_executed if (Environment.status_node_4_put = executed);
  node_5_withdraw_is_disabled if (Environment.status_node_5_withdraw = disabled);
  node_5_withdraw_is_enabled if (Environment.status_node_5_withdraw = enabled);
  node_5_withdraw_is_executed if (Environment.status_node_5_withdraw = executed);
  node_7_split_is_disabled if (Environment.status_node_7_split = disabled);
  node_7_split_is_enabled if (Environment.status_node_7_split = enabled);
  node_7_split_is_executed if (Environment.status_node_7_split = executed);
  contract_funds_are_zero if (Environment.contract_funds = 0);
  part_A_total_deposits_is_10 if (Environment.part_A_total_deposits >= 10);
  part_B_total_deposits_is_at_least_9 if (Environment.part_B_total_deposits >= 9);
end Evaluation
InitStates
  (((((((((((((((((((((Environment.time = 0) and (Environment.part_G_is_done = false)) and (Environment.part_A_is_done = false)) and (Environment.part_B_is_done = false)) and (Environment.contract_funds = 19)) and (Environment.part_G_total_deposits = 0)) and (Environment.part_A_total_deposits = 0)) and (Environment.part_B_total_deposits = 10)) and (Environment.spent_deposit_txb1 = false)) and (Environment.contract_initialized = false)) and (Environment.status_node_0_withdraw = disabled)) and (Environment.status_node_1_withdraw = disabled)) and (Environment.status_node_2_withdraw = disabled)) and (Environment.status_node_3_split = disabled)) and (Environment.status_node_4_put = disabled)) and (Environment.status_node_5_withdraw = disabled)) and (Environment.status_node_7_split = disabled)) and (Environment.last_action = unset)) and (Agent_G.dummy = false)) and (Agent_A.dummy = false)) and (Agent_B.dummy = false));
end InitStates
Groups
  Participants = {Agent_A, Agent_B, Agent_G};
  Env = {Environment};
  ParticipantsAndEnv = {Agent_A, Agent_B, Agent_G, Environment};
  Agent_G = {Agent_G};
  Agent_A = {Agent_A};
  Agent_B = {Agent_B};
  Agent_A__Agent_G = {Agent_A, Agent_G};
  Agent_B__Agent_G = {Agent_B, Agent_G};
  Agent_A__Agent_B = {Agent_A, Agent_B};
end Groups
Fairness
  part_G_is_scheduled;
  part_A_is_scheduled;
  part_B_is_scheduled;
  time_progresses_forever;
end Fairness
Formulae
  (<Agent_A>F(contract_funds_are_zero));
  (<Agent_B>F(contract_funds_are_zero));
  (<Agent_G>F(contract_funds_are_zero));
  (<Agent_A__Agent_G>F(part_A_total_deposits_is_10));
  (<Agent_B>F(part_B_total_deposits_is_at_least_9));
end Formulae
```


By calling MCMAS with the command:

```
./mcmas -atlk 1  docs/04-zero-coupon-bond/contract.ispl
```

We get the following output:

```
************************************************************************
                       MCMAS v1.3.0 

 This software comes with ABSOLUTELY NO WARRANTY, to the extent
 permitted by applicable law. 

 Please check http://vas.doc.ic.ac.uk/tools/mcmas/ for the latest release.
 Please send any feedback to <mcmas@imperial.ac.uk>
************************************************************************

Command line: ./tests/bin/mcmas -atlk 1 docs/04-zero-coupon-bond/contract.ispl

docs/04-zero-coupon-bond/contract.ispl has been parsed successfully.
Global syntax checking...
1
1
1
Done
Encoding BDD parameters...
Building partial transition relation...
Building BDD for initial states...
Building reachable state space...
Checking formulae...
Building set of fair states...
Verifying properties...
  Formula number 1: (<Agent_A>F contract_funds_are_zero), is TRUE in the model
  Formula number 2: (<Agent_B>F contract_funds_are_zero), is TRUE in the model
  Formula number 3: (<Agent_G>F contract_funds_are_zero), is TRUE in the model
  Formula number 4: (<Agent_A__Agent_G>F part_A_total_deposits_is_10), is TRUE in the model
  Formula number 5: (<Agent_B>F part_B_total_deposits_is_at_least_9), is TRUE in the model
done, 5 formulae successfully read and checked
execution time = 4.988
number of reachable states = 1129
BDD memory in use = 28123696
```